## **Featurestore Terminology and Concepts**

### Featurestore Data model

Feature Store organizes data with the following 3 important hierarchical concepts:
```
Featurestore -> EntityType -> Feature
```
* **Featurestore**: the place to store your features
* **EntityType**: under a Featurestore, an *EntityType* describes an object to be modeled, real one or virtual one.
* **Feature**: under an EntityType, a *feature* describes an attribute of the EntityType

In [1]:
# User Inputs / Configuration

PROJECT_ID = "mg-ce-demos"
API_ENDPOINT = "us-central1-aiplatform.googleapis.com" 
INPUT_CSV_FILE = "gs://cloud-samples-data-us-central1/ai-platform-unified/datasets/featurestore/movie_prediction.csv"

DESTINATION_DATA_SET = "movie_predictions"  
DESTINATION_TABLE_NAME = "fs_movie_training_data"
REGION = "us-central1"

### View movie dataset (movie_prediction.csv)

In [2]:
!gsutil cp $INPUT_CSV_FILE . 



Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

Copying gs://cloud-samples-data-us-central1/ai-platform-unified/datasets/featurestore/movie_prediction.csv...
/ [1 files][  255.0 B/  255.0 B]                                                
Operation completed over 1 objects/255.0 B.                                      


In [3]:
!head movie_prediction.csv

users,movies,timestamp
"alice","movie_01",2021-04-15T08:28:14Z
"bob","movie_02",2021-04-15T08:28:14Z
"dav","movie_03",2021-04-15T08:28:14Z
"eve","movie_04",2021-04-15T08:28:14Z
"alice","movie_03",2020-02-14T09:35:15Z
"bob","movie_04",2020-02-14T09:35:15Z


### Import dependencies

In [4]:
import os
import sys
from datetime import datetime

from google.cloud import bigquery
from google.cloud.aiplatform_v1beta1 import (FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient)
from google.cloud.aiplatform_v1beta1.types import FeatureSelector, IdMatcher
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import feature as feature_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_monitoring as featurestore_monitoring_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.protobuf.duration_pb2 import Duration

# Create admin_client for CRUD and data_client for reading feature values.
admin_client = FeaturestoreServiceClient(client_options={"api_endpoint": API_ENDPOINT})
data_client  = FeaturestoreOnlineServingServiceClient(client_options={"api_endpoint": API_ENDPOINT})

# Represents featurestore resource path.
BASE_RESOURCE_PATH = admin_client.common_location_path(PROJECT_ID, REGION)

### Create BigQuery dataset for output

You need a BigQuery dataset to host the output data in `us-central1`. Input the name of the dataset you want to created and specify the name of the table you want to store the output later. These will be used later in the notebook.

**Make sure that the table name does NOT already exist**.

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DESTINATION_DATA_SET = "{prefix}_{timestamp}".format(
    prefix=DESTINATION_DATA_SET, timestamp=TIMESTAMP
)

DESTINATION_PATTERN = "bq://{project}.{dataset}.{table}"
DESTINATION_TABLE_URI = DESTINATION_PATTERN.format(
    project=PROJECT_ID, dataset=DESTINATION_DATA_SET, table=DESTINATION_TABLE_NAME
)

# Create dataset
client = bigquery.Client()
dataset_id = "{}.{}".format(client.project, DESTINATION_DATA_SET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = client.create_dataset(dataset, timeout=30)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset mg-ce-demos.movie_predictions_20220623150518


### Create Featurestore

The method to create a featurestore returns a
[long-running operation](https://google.aip.dev/151) (LRO). An LRO starts an asynchronous job. LROs are returned for other API
methods too, such as updating or deleting a featurestore. Calling
`create_fs_lro.result()` waits for the LRO to complete.

In [6]:
FEATURESTORE_ID = "movie_prediction_{timestamp}".format(timestamp=TIMESTAMP)

create_lro = admin_client.create_featurestore(
    featurestore_service_pb2.CreateFeaturestoreRequest(
        parent=BASE_RESOURCE_PATH,
        featurestore_id=FEATURESTORE_ID,
        featurestore=featurestore_pb2.Featurestore(
            name="Featurestore for movie prediction",
            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                fixed_node_count=3
            ),
        ),
    )
)

In [7]:
# Wait for LRO to finish and get the LRO result.
print(create_lro.result())

name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518"



In [8]:
# Use GetFeaturestore or ListFeaturestores to check if the Featurestore was successfully created. 
# The following example gets the details of the Featurestore.

admin_client.get_featurestore(
    name=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID)
)

name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518"
create_time {
  seconds: 1656011146
  nanos: 363233000
}
update_time {
  seconds: 1656011146
  nanos: 488625000
}
etag: "AMEw9yOnxbqah_nqKDkEIOyWan9CVC1QfOBwWnQgfkynNZ2OpkWdEQK2fYxbimnZImDP"
online_serving_config {
  fixed_node_count: 3
}
state: STABLE

### Create Entity Type
You can specify a monitoring config which will by default be inherited by all Features under this EntityType.

In [9]:
# Create users entity type with monitoring enabled.
# All Features belonging to this EntityType will by default inherit the monitoring config.
users_entity_type_lro = admin_client.create_entity_type(
    featurestore_service_pb2.CreateEntityTypeRequest(
        parent=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id="users",
        entity_type=entity_type_pb2.EntityType(
            description="Users entity",
            monitoring_config=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                snapshot_analysis=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                    monitoring_interval=Duration(seconds=86400),  # 1 day
                ),
            ),
        ),
    )
)

# Similarly, wait for EntityType creation operation.
print(users_entity_type_lro.result())

name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/users"



In [10]:
# Create movies entity type without a monitoring configuration.
movies_entity_type_lro = admin_client.create_entity_type(
    featurestore_service_pb2.CreateEntityTypeRequest(
        parent=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id="movies",
        entity_type=entity_type_pb2.EntityType(description="Movies entity"),
    )
)

# Similarly, wait for EntityType creation operation.
print(movies_entity_type_lro.result())

name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/movies"



### Create Feature

You can also set a custom monitoring configuration at the Feature level, and view the properties and metrics in the console: sample [properties](https://storage.googleapis.com/cloud-samples-data/ai-platform-unified/datasets/featurestore/Feature%20Properties.png), sample [metrics](https://storage.googleapis.com/cloud-samples-data/ai-platform-unified/datasets/featurestore/Feature%20Snapshot%20Distribution.png).

In [11]:
# Create features for the 'users' entity.
# 'age' Feature leaves the monitoring config unset, which means it'll inherit the config from EntityType.
# 'gender' Feature explicitly disables monitoring.
# 'liked_genres' Feature is a STRING_ARRAY type, so it is automatically excluded from monitoring.
# For Features with monitoring enabled, distribution statistics are updated periodically in the console.
admin_client.batch_create_features(
    parent=admin_client.entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, "users"),
    requests=[
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.INT64,
                description="User age",
            ),
            feature_id="age",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="User gender",
                monitoring_config=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                    snapshot_analysis=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                        disabled=True,
                    ),
                ),
            ),
            feature_id="gender",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING_ARRAY,
                description="An array of genres that this user liked",
            ),
            feature_id="liked_genres",
        ),
    ],
).result()

features {
  name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/users/features/age"
}
features {
  name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/users/features/gender"
}
features {
  name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/users/features/liked_genres"
}

In [12]:
# Create features for movies type.
# 'title' Feature enables monitoring.
admin_client.batch_create_features(
    parent=admin_client.entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, "movies"),
    requests=[
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="The title of the movie",
                monitoring_config=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                    snapshot_analysis=featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                        monitoring_interval=Duration(seconds=172800),  # 2 days
                    ),
                ),
            ),
            feature_id="title",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.STRING,
                description="The genres of the movie",
            ),
            feature_id="genres",
        ),
        featurestore_service_pb2.CreateFeatureRequest(
            feature=feature_pb2.Feature(
                value_type=feature_pb2.Feature.ValueType.DOUBLE,
                description="The average rating for the movie, range is [1.0-5.0]",
            ),
            feature_id="average_rating",
        ),
    ],
).result()

features {
  name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/movies/features/title"
}
features {
  name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/movies/features/genres"
}
features {
  name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/movies/features/average_rating"
}

## Search created features

While the [ListFeatures](https://cloud.google.com/vertex-ai/featurestore/docs/reference/rpc/google.cloud.aiplatform.v1beta1#google.cloud.aiplatform.v1beta1.FeaturestoreService.ListFeatures) method allows you to easily view all features of a single
entity type, the [SearchFeatures](https://cloud.google.com/vertex-ai/featurestore/docs/reference/rpc/google.cloud.aiplatform.v1beta1#google.cloud.aiplatform.v1beta1.FeaturestoreService.SearchFeatures) method searches across all featurestores and entity types in a given location (such as `us-central1`). This can help you discover features that were created by someone else.

You can query based on feature properties including feature ID, entity type ID, and feature description. You can also limit results by filtering on a specific featurestore, feature value type, and/or labels.

In [13]:
# Search for all features across all featurestores.
list(admin_client.search_features(location=BASE_RESOURCE_PATH))

[name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/average_rating"
 description: "The average rating for the movie, range is [1.0-5.0]"
 value_type: DOUBLE
 create_time {
   seconds: 1654109215
   nanos: 491912000
 }
 update_time {
   seconds: 1654109215
   nanos: 491912000
 },
 name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/genres"
 description: "The genre of the movie"
 value_type: STRING
 create_time {
   seconds: 1654109215
   nanos: 490144000
 }
 update_time {
   seconds: 1654109215
   nanos: 490144000
 },
 name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/title"
 description: "The title of the movie"
 value_type: STRING
 create_time {
   seconds: 1654109215
   nanos: 488410000
 }
 update_time {
   seconds: 1654109215
   nanos: 488410000
 },
 name: "projects/633472233130/locations/us-central1/features

In [14]:
# Additional search with filter for Double
# Search for all features with value type `DOUBLE`
list(
    admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=BASE_RESOURCE_PATH, query="value_type=DOUBLE"
        )
    )
)

[name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/average_rating"
 description: "The average rating for the movie, range is [1.0-5.0]"
 value_type: DOUBLE
 create_time {
   seconds: 1654109215
   nanos: 491912000
 }
 update_time {
   seconds: 1654109215
   nanos: 491912000
 },
 name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/movies/features/average_rating"
 description: "The average rating for the movie, range is [1.0-5.0]"
 value_type: DOUBLE
 create_time {
   seconds: 1656011198
   nanos: 426503000
 }
 update_time {
   seconds: 1656011198
   nanos: 426503000
 }]

In [15]:
# Additional search with filter on feature value type and keywords.
list(
    admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=BASE_RESOURCE_PATH, query="feature_id:title AND value_type=STRING"
        )
    )
)

[name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/title"
 description: "The title of the movie"
 value_type: STRING
 create_time {
   seconds: 1654109215
   nanos: 488410000
 }
 update_time {
   seconds: 1654109215
   nanos: 488410000
 },
 name: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/movies/features/title"
 description: "The title of the movie"
 value_type: STRING
 create_time {
   seconds: 1656011198
   nanos: 422918000
 }
 update_time {
   seconds: 1656011198
   nanos: 422918000
 }]

## Import Feature Values

You need to import feature values before you can use them for online/offline serving. In this step, you will learn how to import feature values by calling the ImportFeatureValues API using the Python SDK.


In [16]:
import_users_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=admin_client.entity_type_path(
        PROJECT_ID, REGION, FEATURESTORE_ID, "users"
    ),
    avro_source=io_pb2.AvroSource(
        # Source
        gcs_source=io_pb2.GcsSource(
            uris=[
                "gs://cloud-samples-data-us-central1/ai-platform-unified/datasets/featurestore/users.avro"
            ]
        )
    ),
    entity_id_field="user_id",
    feature_specs=[
        # Features
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="age"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="gender"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="liked_genres"),
    ],
    feature_time_field="update_time",
    worker_count=10,
)

In [17]:
# Start to import, will take a couple of minutes
ingestion_lro = admin_client.import_feature_values(import_users_request)

In [18]:
# Polls for the LRO status and prints when the LRO has completed
ingestion_lro.result()

imported_entity_count: 7
imported_feature_value_count: 12

### Import feature values for Movies

Similarly, import feature values for 'movies' into the featurestore.


In [19]:
import_movie_request = featurestore_service_pb2.ImportFeatureValuesRequest(
    entity_type=admin_client.entity_type_path(
        PROJECT_ID, REGION, FEATURESTORE_ID, "movies"
    ),
    avro_source=io_pb2.AvroSource(
        gcs_source=io_pb2.GcsSource(
            uris=[
                "gs://cloud-samples-data-us-central1/ai-platform-unified/datasets/featurestore/movies.avro"
            ]
        )
    ),
    entity_id_field="movie_id",
    feature_specs=[
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="title"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="genres"),
        featurestore_service_pb2.ImportFeatureValuesRequest.FeatureSpec(id="average_rating"),
    ],
    feature_time_field="update_time",
    worker_count=10,
)

In [20]:
# Start to import, will take a couple of minutes
ingestion_lro = admin_client.import_feature_values(import_movie_request)

In [21]:
# Polls for the LRO status and prints when the LRO has completed
ingestion_lro.result()

imported_entity_count: 4
imported_feature_value_count: 12

## Online serving

The
[Online Serving APIs](https://cloud.google.com/vertex-ai/featurestore/docs/reference/rpc/google.cloud.aiplatform.v1beta1#featurestoreonlineservingservice)
lets you serve feature values for small batches of entities. It's designed for latency-sensitive service, such as online model prediction. For example, for a movie service, you might want to quickly shows movies that the current user would most likely watch by using online predictions.

In [22]:
# Fetch the following 3 features.
feature_selector = FeatureSelector(
    id_matcher=IdMatcher(ids=["age", "gender", "liked_genres"])
)

data_client.read_feature_values(
    featurestore_online_service_pb2.ReadFeatureValuesRequest(
        # Fetch from the following feature store/entity type
        entity_type=admin_client.entity_type_path(
            PROJECT_ID, REGION, FEATURESTORE_ID, "users"
        ),
        # Fetch the user features whose ID is "alice"
        entity_id="alice",
        feature_selector=feature_selector,
    )
)

header {
  entity_type: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/users"
  feature_descriptors {
    id: "age"
  }
  feature_descriptors {
    id: "gender"
  }
  feature_descriptors {
    id: "liked_genres"
  }
}
entity_view {
  entity_id: "alice"
  data {
  }
  data {
  }
  data {
  }
}

### Read multiple entities per request

To read feature values from multiple entities, use the
StreamingReadFeatureValues API, which is almost identical to the previous
ReadFeatureValues API. Note that fetching only a small number of entities is recomended when using this API due to its latency-sensitive nature.

In [23]:
# Read the same set of features as above, but for multiple entities.
response_stream = data_client.streaming_read_feature_values(
    featurestore_online_service_pb2.StreamingReadFeatureValuesRequest(
        entity_type=admin_client.entity_type_path(
            PROJECT_ID, REGION, FEATURESTORE_ID, "users"
        ),
        entity_ids=["alice", "bob"],
        feature_selector=feature_selector,
    )
)

In [24]:
# Iterate and process response. Note the first one is always the header only.
for response in response_stream:
    print(response)

header {
  entity_type: "projects/633472233130/locations/us-central1/featurestores/movie_prediction_20220623150518/entityTypes/users"
  feature_descriptors {
    id: "age"
  }
  feature_descriptors {
    id: "gender"
  }
  feature_descriptors {
    id: "liked_genres"
  }
}

entity_view {
  entity_id: "alice"
  data {
  }
  data {
  }
  data {
  }
}

entity_view {
  entity_id: "bob"
  data {
  }
  data {
  }
  data {
  }
}



Now that you have learned how to featch imported feature values for online serving, the next step is learning how to use imported feature values for offline use cases.

## Batch Serving

Batch Serving is used to fetch a large batch of feature values for high-throughput, typically for training a model or batch prediction. In this section, you will learn how to prepare for training examples by calling the BatchReadFeatureValues API.

### Batch Read Feature Values

Assemble the request which specify the following info:

*   Where is the label data, i.e., Table 1.
*   Which features are read, i.e., the column names in Table 2.

The output is stored in a BigQuery table.


In [25]:
batch_serving_request = featurestore_service_pb2.BatchReadFeatureValuesRequest(
    
    # featurestore info
    featurestore=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
    
    # URL for the label data (i.e. Table 1)
    csv_read_instances=io_pb2.CsvSource(
        gcs_source=io_pb2.GcsSource(uris=[INPUT_CSV_FILE])
    ),
    destination=featurestore_service_pb2.FeatureValueDestination(
        bigquery_destination=io_pb2.BigQueryDestination(
            # Output to BigQuery table created earlier
            output_uri=DESTINATION_TABLE_URI
        )
    ),
    
    entity_type_specs=[
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # Read the 'age', 'gender' and 'liked_genres' features from the 'users' entity
            entity_type_id="users",
            feature_selector=FeatureSelector(
                id_matcher=IdMatcher(
                    ids=[
                        # features, use "*" if you want to select all features within this entity type
                        "age",
                        "gender",
                        "liked_genres",
                    ]
                )
            ),
        ),
        featurestore_service_pb2.BatchReadFeatureValuesRequest.EntityTypeSpec(
            # Read the 'average_rating' and 'genres' feature values of the 'movies' entity
            entity_type_id="movies",
            feature_selector=FeatureSelector(
                id_matcher=IdMatcher(ids=["average_rating", "genres"])
            ),
        ),
    ],
)

In [26]:
# Execute the batch read
batch_serving_lro = admin_client.batch_read_feature_values(batch_serving_request)

In [27]:
# This long runing operation will poll until the batch read finishes.
batch_serving_lro.result()

After the LRO finishes, you should be able to see the result from the [BigQuery console](https://console.cloud.google.com/bigquery), in the dataset created earlier.

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

You can also keep the project but delete the featurestore:

In [ ]:
admin_client.delete_featurestore(
    request=featurestore_service_pb2.DeleteFeaturestoreRequest(
        name=admin_client.featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        force=True,
    )
).result()
client.delete_dataset(
    DESTINATION_DATA_SET, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(DESTINATION_DATA_SET))